In [1]:
import pandas as pd

json_conversa = {
    "conversa": [
        {
            "quem_fala": "atendente",
            "Frases": "Olá, em que posso ajudá-lo hoje?",
            "data_hora": "2023-04-22 10:00:00"
        },
        {
            "quem_fala": "agente",
            "Frases": "Oi, estou com um problema no meu celular. Ele está superaquecendo e desligando sozinho.",
            "data_hora": "2023-04-22 10:01:00"
        },
        {
            "quem_fala": "atendente",
            "Frases": "Compreendo. Você já tentou reiniciar o seu celular para ver se isso resolve o problema?",
            "data_hora": "2023-04-22 10:02:00"
        },
        {
            "quem_fala": "agente",
            "Frases": "Sim, já tentei reiniciar algumas vezes, mas o problema persiste.",
            "data_hora": "2023-04-22 10:03:00"
        },
        {
            "quem_fala": "atendente",
            "Frases": "Entendo. Pode ser que o superaquecimento seja causado por algum aplicativo ou função que esteja consumindo muita energia. Você poderia me informar quais aplicativos costuma utilizar com mais frequência?",
            "data_hora": "2023-04-22 10:05:00"
        },
        {
            "quem_fala": "agente",
            "Frases": "Sim, costumo usar muito o aplicativo de redes sociais e também assisto a vídeos no meu celular.",
            "data_hora": "2023-04-22 10:06:00"
        },
        {
            "quem_fala": "atendente",
            "Frases": "Ok, nesse caso, sugiro que tente desinstalar alguns aplicativos que você não usa com tanta frequência e também tente evitar usar o celular enquanto ele está carregando. Se o problema persistir, pode ser necessário levá-lo a uma assistência técnica para uma avaliação mais detalhada.",
            "data_hora": "2023-04-22 10:08:00"
        },
        {
            "quem_fala": "agente",
            "Frases": "Certo, vou tentar fazer isso. Obrigado pela ajuda.",
            "data_hora": "2023-04-22 10:09:00"
        },
        {
            "quem_fala": "atendente",
            "Frases": "De nada. Fico feliz em poder ajudar. Qualquer dúvida adicional, não hesite em entrar em contato conosco novamente.",
            "data_hora": "2023-04-22 10:10:00"
        },
        {
            "quem_fala": "agente",
            "Frases": "Certo, vou tentar fazer isso. Obrigado pela ajuda.",
            "data_hora": "2023-04-22 10:09:00"
        },
        {
            "quem_fala": "atendente",
            "Frases": "De nada. Fico feliz em poder ajudar. Qualquer dúvida adicional, não hesite em entrar em contato conosco novamente.",
            "data_hora": "2023-04-22 10:10:00"
        },
        {
            "quem_fala": "agente",
            "Frases": "A propósito, vocês têm alguma recomendação de aplicativo para monitorar a temperatura do celular?",
            "data_hora": "2023-04-22 10:11:00"
        },
        {
            "quem_fala": "atendente",
            "Frases": "Sim, você pode usar aplicativos como 'CPU Cooler Master' ou 'Cooler Master - Phone Cooler'. Eles são úteis para monitorar a temperatura do seu celular e otimizar o desempenho do dispositivo.",
            "data_hora": "2023-04-22 10:13:00"
        },
        {
            "quem_fala": "agente",
            "Frases": "Ótimo, vou dar uma olhada nesses aplicativos. Agradeço novamente pela ajuda.",
            "data_hora": "2023-04-22 10:14:00"
        },
        {
            "quem_fala": "atendente",
            "Frases": "Sem problemas! Estamos aqui para ajudar. Se precisar de mais informações ou tiver mais dúvidas, sinta-se à vontade para nos contatar. Tenha um ótimo dia!",
            "data_hora": "2023-04-22 10:15:00"
        }
    ]
}

df = pd.DataFrame(json_conversa["conversa"], columns=["quem_fala", "Frases", "data_hora"])


A classe SentimentAnalyzer realiza a análise de sentimentos de um texto utilizando modelos de Processamento de Linguagem Natural (PLN) pré-treinados, implementados pela biblioteca transformers do Python. Ela permite rotular um texto como muito negativo, negativo, neutro, positivo ou muito positivo, adicionando essa informação a um dataframe de frases fornecido pelo usuário. Além disso, a classe permite a criação de diversas visualizações gráficas que exploram as informações contidas no dataframe, como a distribuição de sentimentos nas frases, a polaridade das frases, a relação entre a polaridade e o tamanho das frases, a relação entre a categoria de sentimento e o número de palavras nas frases, entre outras. A classe utiliza outras bibliotecas, como pandas, seaborn, matplotlib e textblob.


In [2]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob


class SentimentAnalyzer:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.sentiment_names = ["muito negativo", "negativo", "neutro", "positivo", "muito positivo"]

    def label_sentiment(self, text):
        inputs = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        outputs = self.model(**inputs)
        predicted_class = torch.argmax(outputs.logits)
        return self.sentiment_names[predicted_class.item()]

    def analyze_sentiments(self, df, text_column="Frases", label_column="Sentimento"):
        df[label_column] = df[text_column].apply(self.label_sentiment)
        return df

    def add_length_and_polarity_columns(self, df):
        df["Tamanho"] = df["Frases"].apply(lambda x: len(x.split()))
        df["Polaridade"] = df["Frases"].apply(lambda x: TextBlob(x).sentiment.polarity)
        return df

    def add_word_count_column(self, df):
        df["NumPalavras"] = df["Frases"].apply(lambda x: len(x.split()))
        return df

    def add_category_column(self, df):
        df["Categoria"] = df["Sentimento"].apply(lambda x: "detrator" if x in ["muito negativo", "negativo"]
                                                 else ("promotor" if x in ["positivo", "muito positivo"]
                                                 else "neutro"))
        return df

    def plot_sentiment_distribution(self, df):
        sns.histplot(df["Sentimento"], color="red", alpha=0.7)
        sns.set_style("whitegrid")
        plt.title("Distribuição de sentimentos nas frases rotuladas")
        plt.xlabel("Sentimento")
        plt.ylabel("Número de frases")
        plt.show()

    def plot_sentiment_pie(self, df):
        counts = df["Sentimento"].value_counts()
        explode = [0.1] * len(counts) # Lista com o mesmo tamanho do número de valores únicos na coluna "Sentimento"
        counts.plot(kind="pie", autopct="%1.1f%%", startangle=90, 
                    explode=explode, wedgeprops={'linewidth': 1, 'edgecolor': 'black'})
        plt.title("Proporção de sentimentos nas frases rotuladas")
        plt.legend(bbox_to_anchor=(1, 0.7))
        plt.show()


    def plot_sentiment_bar(self, df):
        counts = df["Sentimento"].value_counts().sort_index()
        sns.barplot(x=counts.index, y=counts.values, palette="rocket")
        plt.title("Contagem de sentimentos nas frases rotuladas")
        plt.xlabel("Sentimento")
        plt.ylabel("Número de frases")
        plt.show()

    def plot_sentiment_radar(self, df):
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(polar=True)
        categories = self.sentiment_names
        angles = np.linspace(0, 2 * np.pi, len(categories) + 1, endpoint=True)
        values = df["Sentimento"].value_counts(normalize=True).sort_index().tolist()
        while len(values) < len(categories):
            values.insert(categories.index("neutro"), 0)
        values.append(values[0])
        ax.plot(angles, values, 'o-', linewidth=2, label="Frases")
        ax.fill(angles, values, alpha=0.25)
        ax.set_thetagrids(np.degrees(angles[:-1]), categories)
        ax.set_title("Frequência relativa de sentimentos")
        plt.legend()
        plt.show()


    def plot_polarity_histogram(self, df):
        sns.histplot(df["Polaridade"], color="blue", alpha=0.7)
        sns.set_style("whitegrid")
        plt.title("Distribuição de polaridade nas frases")
        plt.xlabel("Polaridade")
        plt.ylabel("Número de frases")
        plt.show()

    def plot_polarity_vs_length_scatter(self, df):
        sns.scatterplot(x=df["Tamanho"], y=df["Polaridade"], color="blue", alpha=0.7)
        plt.title("Relação entre polaridade e tamanho da frase")
        plt.xlabel("Tamanho da frase")
        plt.ylabel("Polaridade")
        plt.show()

    def plot_polarity_density(self, df):
        sns.displot(data=df, x="Tamanho", y="Polaridade", kind="kde", fill=True, thresh=0, levels=100, cmap="coolwarm")
        plt.title("Distribuição de polaridade por tamanho de frase")
        plt.xlabel("Tamanho da frase")
        plt.ylabel("Polaridade")
        plt.show()
        
    def plot_category_bar(self, df):
        counts = df["Categoria"].value_counts().sort_index()
        sns.barplot(x=counts.index, y=counts.values, palette="viridis")
        plt.title("Contagem de categorias nas frases rotuladas")
        plt.xlabel("Categoria")
        plt.ylabel("Número de frases")
        plt.show()

    def plot_category_vs_word_count_scatter(self, df):
        sns.scatterplot(x=df["NumPalavras"], y=df["Categoria"], alpha=0.7)
        plt.title("Relação entre número de palavras e categoria de sentimento")
        plt.xlabel("Número de palavras")
        plt.ylabel("Categoria")
        plt.show()
        
    def plot_all_visualizations(self, df):
        self.plot_sentiment_distribution(df)
        self.plot_sentiment_pie(df)
        self.plot_sentiment_bar(df)
        self.plot_sentiment_radar(df)
        self.plot_polarity_histogram(df)
        self.plot_polarity_vs_length_scatter(df)
        self.plot_polarity_density(df)
        self.plot_category_bar(df)
        self.plot_category_vs_word_count_scatter(df)
        
    def add_all_columns(self, df):
        df["Tamanho"] = df["Frases"].apply(lambda x: len(x.split()))
        df["Polaridade"] = df["Frases"].apply(lambda x: TextBlob(x).sentiment.polarity)
        df["NumPalavras"] = df["Frases"].apply(lambda x: len(x.split()))
        df["Categoria"] = df["Sentimento"].apply(lambda x: "detrator" if x in ["muito negativo", "negativo"]
                                                 else ("promotor" if x in ["positivo", "muito positivo"]
                                                 else "neutro"))
        return df





C:\Users\santo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Instanciando a classe SentimentAnalyzer com o modelo pré-treinado "nlptown/bert-base-multilingual-uncased-sentiment"
analyzer = SentimentAnalyzer("nlptown/bert-base-multilingual-uncased-sentiment")

# Analisando os sentimentos das frases
df1 = analyzer.analyze_sentiments(df)

# Adicionando todas as colunas
df1 = analyzer.add_all_columns(df1)

# Gerando todas as visualizações
#analyzer.plot_all_visualizations(df)



In [4]:
df1

,quem_fala,Frases,data_hora,Sentimento,Tamanho,Polaridade,NumPalavras,Categoria
0,atendente,"Olá, em que posso ajudá-lo hoje?",2023-04-22 10:00:00,neutro,6,0.0,6,neutro
1,agente,"Oi, estou com um problema no meu celular. Ele ...",2023-04-22 10:01:00,muito negativo,14,0.0,14,detrator
2,atendente,Compreendo. Você já tentou reiniciar o seu cel...,2023-04-22 10:02:00,neutro,15,0.0,15,neutro
3,agente,"Sim, já tentei reiniciar algumas vezes, mas o ...",2023-04-22 10:03:00,neutro,10,0.0,10,neutro
4,atendente,Entendo. Pode ser que o superaquecimento seja ...,2023-04-22 10:05:00,negativo,29,0.0,29,detrator
5,agente,"Sim, costumo usar muito o aplicativo de redes ...",2023-04-22 10:06:00,neutro,17,0.0,17,neutro
6,atendente,"Ok, nesse caso, sugiro que tente desinstalar a...",2023-04-22 10:08:00,neutro,44,0.5,44,neutro
7,agente,"Certo, vou tentar fazer isso. Obrigado pela aj...",2023-04-22 10:09:00,neutro,8,0.0,8,neutro
8,atendente,De nada. Fico feliz em poder ajudar. Qualquer ...,2023-04-22 10:10:00,muito negativo,18,0.0,18,detrator
9,agente,"Certo, vou tentar fazer isso. Obrigado pela aj...",2023-04-22 10:09:00,neutro,8,0.0,8,neutro


A classe AnalisadorDeEmocoes é uma classe que utiliza um modelo pré-treinado de classificação de emoções em texto, baseado no transformer twitter-roberta-base. Ela oferece diversos métodos para análise de emoções em textos, como a rotulagem de emoções em um DataFrame, a tradução de emoções para o português, a geração de n-gramas para análise de padrões, e a visualização de gráficos relacionando emoção e polaridade.



In [5]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from googletrans import Translator

class AnalisadorDeEmocoes:
    def __init__(self, model_name="cardiffnlp/twitter-roberta-base-emotion"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.translator = Translator()
        self.emotion_names = ["raiva", "medo", "alegria", "tristeza", "surpresa", "neutro"]

    def translate_input(self, text):
        translated_text = self.translator.translate(text, src='pt', dest='en').text
        return translated_text

    def translate_output(self, emotion):
        translated_emotion = self.translator.translate(emotion, src='en', dest='pt').text
        return translated_emotion

    def label_emotion(self, text):
        translated_text = self.translate_input(text)
        inputs = self.tokenizer(translated_text, padding=True, truncation=True, return_tensors="pt")
        outputs = self.model(**inputs)
        predicted_class = torch.argmax(outputs.logits)
        translated_emotion = self.translate_output(self.emotion_names[predicted_class.item()])
        return translated_emotion

    def analyze_emotions(self, df, text_column="Frases", label_column="Emoções"):
        df[label_column] = df[text_column].apply(self.label_emotion)
        return df
    
    def add_translated_emotion_column(self, df, text_column="Frases", translated_emotion_column="Emoção traduzida"):
        df[translated_emotion_column] = df[text_column].apply(self.translate_output)
        return df
    
    def generate_ngrams(self, text, n=2):
        blob = TextBlob(text)
        ngrams = blob.ngrams(n)
        return [' '.join(gram) for gram in ngrams]

    def add_ngrams_column(self, df, text_column="Frases", ngrams_column="N-grams", n=2):
        df[ngrams_column] = df[text_column].apply(lambda x: self.generate_ngrams(x, n))
        return df
    
    def plot_emotion_vs_polarity_scatter(self, df, sentiment_analyzer):
        df["Polaridade"] = df["Frases"].apply(lambda x: TextBlob(x).sentiment.polarity)
        sns.scatterplot(x=df["Emoções"], y=df["Polaridade"], alpha=0.7)
        plt.title("Relação entre emoção e polaridade")
        plt.xlabel("Emoção")
        plt.ylabel("Polaridade")
        plt.show()


    def plot_emotion_distribution(self, df):
        sns.histplot(df["Emoções"], color="red", alpha=0.7)
        sns.set_style("whitegrid")
        plt.title("Distribuição de emoções nas frases rotuladas")
        plt.xlabel("Emoção")
        plt.ylabel("Número de frases")
        plt.show()

    def plot_emotion_pie(self, df):
        counts = df["Emoções"].value_counts()
        explode = [0.1] * len(counts)
        counts.plot(kind="pie", autopct="%1.1f%%", startangle=90, 
                    explode=explode, wedgeprops={'linewidth': 1, 'edgecolor': 'black'})
        plt.title("Proporção de emoções nas frases rotuladas")
        plt.legend(bbox_to_anchor=(1, 0.7))
        plt.show()

    def plot_emotion_bar(self, df):
        counts = df["Emoções"].value_counts().sort_index()
        sns.barplot(x=counts.index, y=counts.values, palette="rocket")
        plt.title("Contagem de emoções nas frases rotuladas")
        plt.xlabel("Emoção")
        plt.ylabel("Número de frases")
        plt.show()

    def plot_all_visualizations(self, df):
        self.plot_emotion_distribution(df)
        self.plot_emotion_pie(df)
        self.plot_emotion_bar(df)


In [6]:
import nltk
nltk.download('punkt')

analisador = AnalisadorDeEmocoes()
df2 = analisador.analyze_emotions(df)
df2 = analisador.add_translated_emotion_column(df2)
df2 = analisador.add_ngrams_column(df2, n=2)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
df2

,quem_fala,Frases,data_hora,Sentimento,Tamanho,Polaridade,NumPalavras,Categoria,Emoções,Emoção traduzida,N-grams
0,atendente,"Olá, em que posso ajudá-lo hoje?",2023-04-22 10:00:00,neutro,6,0.0,6,neutro,Tristeza,"Olá, em que poder ajudá-lo hoje?","[Olá em, em que, que posso, posso ajudá-lo, aj..."
1,agente,"Oi, estou com um problema no meu celular. Ele ...",2023-04-22 10:01:00,muito negativo,14,0.0,14,detrator,Tristeza,"Oi, estou com problemas sem meu celular.Ele Es...","[Oi estou, estou com, com um, um problema, pro..."
2,atendente,Compreendo. Você já tentou reiniciar o seu cel...,2023-04-22 10:02:00,neutro,15,0.0,15,neutro,Tristeza,Compreendo.Você Já Tentou reiniciar o Seu Celu...,"[Compreendo Você, Você já, já tentou, tentou r..."
3,agente,"Sim, já tentei reiniciar algumas vezes, mas o ...",2023-04-22 10:03:00,neutro,10,0.0,10,neutro,Tristeza,"Sim, Já Tentei Reiniciar Algumas Vezes, Mas o ...","[Sim já, já tentei, tentei reiniciar, reinicia..."
4,atendente,Entendo. Pode ser que o superaquecimento seja ...,2023-04-22 10:05:00,negativo,29,0.0,29,detrator,Tristeza,Entendo.PODE SER QUE O OURAQUECIMENTO SEJA CAU...,"[Entendo Pode, Pode ser, ser que, que o, o sup..."
5,agente,"Sim, costumo usar muito o aplicativo de redes ...",2023-04-22 10:06:00,neutro,17,0.0,17,neutro,Medo,"Sim, costumo usar muito oplicativo de Redes So...","[Sim costumo, costumo usar, usar muito, muito ..."
6,atendente,"Ok, nesse caso, sugiro que tente desinstalar a...",2023-04-22 10:08:00,neutro,44,0.5,44,neutro,Tristeza,"OK, NESSE CASO, SUGIRO Que TEDE DESINSTALAR AL...","[Ok nesse, nesse caso, caso sugiro, sugiro que..."
7,agente,"Certo, vou tentar fazer isso. Obrigado pela aj...",2023-04-22 10:09:00,neutro,8,0.0,8,neutro,Alegria,"Certo, Vou Tud Fazer ISSO.Obriguado Pela Ajuda.","[Certo vou, vou tentar, tentar fazer, fazer is..."
8,atendente,De nada. Fico feliz em poder ajudar. Qualquer ...,2023-04-22 10:10:00,muito negativo,18,0.0,18,detrator,Alegria,De Nada.Fico Feliz Em Poder Ajudar.Qualquer Dú...,"[De nada, nada Fico, Fico feliz, feliz em, em ..."
9,agente,"Certo, vou tentar fazer isso. Obrigado pela aj...",2023-04-22 10:09:00,neutro,8,0.0,8,neutro,Alegria,"Certo, Vou Tud Fazer ISSO.Obriguado Pela Ajuda.","[Certo vou, vou tentar, tentar fazer, fazer is..."


In [8]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import pandas as pd

class AnalisadorDeSentimentoAvancado:
    def __init__(self):
        pass

    def get_aspect(self, text):
        blob = TextBlob(text)
        return [np for np in blob.noun_phrases]

    def add_aspect_column(self, df, text_column="Frases", aspect_column="Aspectos"):
        df[aspect_column] = df[text_column].apply(lambda x: self.get_aspect(x))
        return df

    def rule_based_sentiment(self, text):
        blob = TextBlob(text, analyzer=NaiveBayesAnalyzer())
        return blob.sentiment.classification

    def add_rule_based_sentiment_column(self, df, text_column="Frases", rule_based_sentiment_column="Sentimento_regras"):
        df[rule_based_sentiment_column] = df[text_column].apply(lambda x: self.rule_based_sentiment(x))
        return df


In [9]:
# criar uma instância da classe AnalisadorDeSentimentoAvancado
analisador = AnalisadorDeSentimentoAvancado()

# adicionar uma coluna de aspectos ao DataFrame
df3 = analisador.add_aspect_column(df)

# adicionar uma coluna de sentimento baseado em regras ao DataFrame
df3 = analisador.add_rule_based_sentiment_column(df3)

# imprimir o DataFrame resultante
print(df3)

    quem_fala                                             Frases   
0   atendente                   Olá, em que posso ajudá-lo hoje?  \
1      agente  Oi, estou com um problema no meu celular. Ele ...   
2   atendente  Compreendo. Você já tentou reiniciar o seu cel...   
3      agente  Sim, já tentei reiniciar algumas vezes, mas o ...   
4   atendente  Entendo. Pode ser que o superaquecimento seja ...   
5      agente  Sim, costumo usar muito o aplicativo de redes ...   
6   atendente  Ok, nesse caso, sugiro que tente desinstalar a...   
7      agente  Certo, vou tentar fazer isso. Obrigado pela aj...   
8   atendente  De nada. Fico feliz em poder ajudar. Qualquer ...   
9      agente  Certo, vou tentar fazer isso. Obrigado pela aj...   
10  atendente  De nada. Fico feliz em poder ajudar. Qualquer ...   
11     agente  A propósito, vocês têm alguma recomendação de ...   
12  atendente  Sim, você pode usar aplicativos como 'CPU Cool...   
13     agente  Ótimo, vou dar uma olhada nesses 

In [10]:
df3

,quem_fala,Frases,data_hora,Sentimento,Tamanho,Polaridade,NumPalavras,Categoria,Emoções,Emoção traduzida,N-grams,Aspectos,Sentimento_regras
0,atendente,"Olá, em que posso ajudá-lo hoje?",2023-04-22 10:00:00,neutro,6,0.0,6,neutro,Tristeza,"Olá, em que poder ajudá-lo hoje?","[Olá em, em que, que posso, posso ajudá-lo, aj...","[olá, em que posso ajudá-lo hoje]",pos
1,agente,"Oi, estou com um problema no meu celular. Ele ...",2023-04-22 10:01:00,muito negativo,14,0.0,14,detrator,Tristeza,"Oi, estou com problemas sem meu celular.Ele Es...","[Oi estou, estou com, com um, um problema, pro...","[oi, estou com um problema, meu celular, ele, ...",neg
2,atendente,Compreendo. Você já tentou reiniciar o seu cel...,2023-04-22 10:02:00,neutro,15,0.0,15,neutro,Tristeza,Compreendo.Você Já Tentou reiniciar o Seu Celu...,"[Compreendo Você, Você já, já tentou, tentou r...","[compreendo, você, já tentou reiniciar o seu c...",pos
3,agente,"Sim, já tentei reiniciar algumas vezes, mas o ...",2023-04-22 10:03:00,neutro,10,0.0,10,neutro,Tristeza,"Sim, Já Tentei Reiniciar Algumas Vezes, Mas o ...","[Sim já, já tentei, tentei reiniciar, reinicia...","[sim, já tentei reiniciar algumas vezes, mas o...",pos
4,atendente,Entendo. Pode ser que o superaquecimento seja ...,2023-04-22 10:05:00,negativo,29,0.0,29,detrator,Tristeza,Entendo.PODE SER QUE O OURAQUECIMENTO SEJA CAU...,"[Entendo Pode, Pode ser, ser que, que o, o sup...","[entendo, pode, ser que o superaquecimento sej...",pos
5,agente,"Sim, costumo usar muito o aplicativo de redes ...",2023-04-22 10:06:00,neutro,17,0.0,17,neutro,Medo,"Sim, costumo usar muito oplicativo de Redes So...","[Sim costumo, costumo usar, usar muito, muito ...","[sim, costumo usar muito o aplicativo, redes s...",pos
6,atendente,"Ok, nesse caso, sugiro que tente desinstalar a...",2023-04-22 10:08:00,neutro,44,0.5,44,neutro,Tristeza,"OK, NESSE CASO, SUGIRO Que TEDE DESINSTALAR AL...","[Ok nesse, nesse caso, caso sugiro, sugiro que...","[ok, nesse caso, sugiro que tente desinstalar ...",neg
7,agente,"Certo, vou tentar fazer isso. Obrigado pela aj...",2023-04-22 10:09:00,neutro,8,0.0,8,neutro,Alegria,"Certo, Vou Tud Fazer ISSO.Obriguado Pela Ajuda.","[Certo vou, vou tentar, tentar fazer, fazer is...","[certo, vou tentar fazer isso, obrigado, pela ...",pos
8,atendente,De nada. Fico feliz em poder ajudar. Qualquer ...,2023-04-22 10:10:00,muito negativo,18,0.0,18,detrator,Alegria,De Nada.Fico Feliz Em Poder Ajudar.Qualquer Dú...,"[De nada, nada Fico, Fico feliz, feliz em, em ...","[de, fico, feliz em poder ajudar, qualquer, dú...",neg
9,agente,"Certo, vou tentar fazer isso. Obrigado pela aj...",2023-04-22 10:09:00,neutro,8,0.0,8,neutro,Alegria,"Certo, Vou Tud Fazer ISSO.Obriguado Pela Ajuda.","[Certo vou, vou tentar, tentar fazer, fazer is...","[certo, vou tentar fazer isso, obrigado, pela ...",pos


In [14]:
df3.columns

Index(['quem_fala', 'Frases', 'data_hora', 'Sentimento', 'Tamanho',
       'Polaridade', 'NumPalavras', 'Categoria', 'Emoções', 'Emoção traduzida',
       'N-grams', 'Aspectos', 'Sentimento_regras'],
      dtype='object')